In [1]:
time_start = 20150101
time_end =  20210630
today = 20211130
feat_list = ['pre_close','open','high','low','close','vwap','volume','pct_chg','turnover','free_turnover','chg']
return_value = '5d_ret'
output_path = '/home/wuwenjun/Data/AlphaNet_Original_Input/'
target = '5d_ret'
day_back = 5 # 回溯几天进行计算
day = 30 # 一次提取几天的数,一般默认为30
stride = 10 # 一次学习多少天

In [2]:
import time
import os
from Research.backtest.bt import BTDaily
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
from Platform.database.mysql import MysqlAPI
from Platform.utils.persistence import convert_to_standard_daily_feature_csv, convert_to_standard_daily_feature_par
from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
import DataAPI
from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm

In [3]:
dataframe_list = DataAPI.get_stock_prices(start_date=time_start, end_date=time_end).reset_index().set_index(['timestamp','ticker'])

In [4]:
dataframe_list = dataframe_list[feat_list].reset_index()
time_df = pd.DataFrame(dataframe_list['timestamp'].drop_duplicates())
time_df['timestamp_shift'] = time_df['timestamp'].apply(lambda x : DataAPI.get_next_trading_day(x, output='datetime'))
dataframe_list = pd.merge(left = dataframe_list , right = time_df , how = 'left',on = 'timestamp')
dataframe_list.drop(['timestamp'],axis=1,inplace=True)
dataframe_list.rename(columns = {"timestamp_shift":"timestamp"},inplace=True)
dataframe_list['timestamp'] = pd.to_datetime(dataframe_list['timestamp']) + pd.Timedelta(hours=9,minutes = 30)
dataframe_list.set_index(['timestamp','ticker'],inplace = True)
dataframe_list

pre_close    open    high     low   close  \
timestamp           ticker                                              
2015-01-06 09:30:00 000001      15.84   15.99   16.28   15.60   16.02   
                    000002      13.90   14.39   15.29   14.22   14.91   
                    000004      15.57   15.58   15.90   15.30   15.69   
                    000005       4.10    4.10    4.10    4.10    4.10   
                    000006       7.05    7.10    7.39    7.02    7.08   
...                               ...     ...     ...     ...     ...   
2021-07-01 09:30:00 688700      46.60   47.80   54.50   47.03   53.38   
                    688777      90.00   89.95   94.80   89.06   94.15   
                    688788     138.20  137.28  148.50  136.33  145.38   
                    688819      43.68   43.40   44.98   43.31   44.49   
                    688981      59.25   59.69   62.50   59.40   61.82   

                                vwap      volume  pct_chg  turnover  \
timestamp           ticker                                            
2015-01-06 09:30:00 000001   15.9605  2860436.43   1.1364    2.9079   
                    000002   14.7858  6560835.70   7.2662    6.7834   
                    000004   15.5709    28095.12   0.7707    3.3496   
                    000005    4.1000        0.00   0.0000    0.0000   
                    000006    7.1881  1078647.12   0.4255    8.0562   
...                              ...         ...      ...       ...   
2021-07-01 09:30:00 688700   50.7153    64336.83  14.5494   19.2222   
                    688777   92.9798    19001.61   4.6111    4.6288   
                    688788  144.3683    13572.56   5.1954    7.7977   
                    688819   44.3727    47249.01   1.8544    4.6754   
                    688981   61.5420   777950.53   4.3376    7.1003   

                            free_turnover   chg  
timestamp           ticker                       
2015-01-06 09:30:00 000001         5.7657  0.18  
                    000002         8.1741  1.01  
                    000004         4.5343  0.12  
                    000005         0.0000  0.00  
                    000006        13.2751  0.03  
...                                   ...   ...  
2021-07-01 09:30:00 688700        19.2222  6.78  
                    688777         4.6288  4.15  
                    688788         7.7977  7.18  
                    688819         4.6754  0.81  
                    688981         7.1003  2.57  

[5357761 rows x 11 columns]

In [5]:
config_path = r'/home/ShareFolder/lgc/Modules/Research/config/feature_bt_template'
print('Loading the configuration from ' + config_path)
configs = namespace.load_namespace(config_path)
FT = FeatureAnalysis(configs, feature_path=r"/home/ShareFolder/factor_lib")

Loading the configuration from /home/ShareFolder/lgc/Modules/Research/config/feature_bt_template


Process ForkPoolWorker-141:
Process ForkPoolWorker-131:
Process ForkPoolWorker-130:
Process ForkPoolWorker-142:
Process ForkPoolWorker-140:
Process ForkPoolWorker-75:
Process ForkPoolWorker-91:
Process ForkPoolWorker-74:
Process ForkPoolWorker-86:
Process ForkPoolWorker-137:
Process ForkPoolWorker-128:
Process ForkPoolWorker-133:
Process ForkPoolWorker-144:
Process ForkPoolWorker-143:
Process ForkPoolWorker-100:
Process ForkPoolWorker-94:
Process ForkPoolWorker-104:
Process ForkPoolWorker-124:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-88:
Process ForkPoolWorker-80:
Traceback (most recent call last):
Process ForkPoolWorker-119:
Process ForkPoolWorker-139:
Process ForkPoolWorker-103:
Process ForkPoolWorker-97:
Process ForkPoolWorker-132:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call 

  File "/usr/local/miniconda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  

In [6]:
FT.load_feature_from_dataframe(dataframe_list,universe='Investable')
FT.load_return_data()

Merging feature and return...: 100%|█████████████████████████████████████████████████████████████████████████████| 1580/1580 [00:08<00:00, 195.78it/s]


In [7]:
data_info = FT.data_info

In [8]:
# Drop 掉不需要的return 列
df = pd.DataFrame()
return_list = feat_list.copy()
return_list.append(return_value)
for timestamp in tqdm(data_info.keys()):
    timestamp = pd.to_datetime(timestamp)
    one_data = data_info[timestamp][return_list].reset_index()
    one_data['timestamp'] = timestamp
    df = pd.concat([df,one_data],axis=0)
df.sort_values(by = ['timestamp','ticker'],inplace=True)
display(df)

100%|██████████| 1580/1580 [05:40<00:00,  4.65it/s]


,ticker,pre_close,open,high,low,close,vwap,volume,pct_chg,turnover,free_turnover,chg,5d_ret,timestamp
0,000001,15.84,15.99,16.28,15.60,16.02,15.9605,2860436.43,1.1364,2.9079,5.7657,0.18,-0.075710,2015-01-06 09:30:00
1,000002,13.90,14.39,15.29,14.22,14.91,14.7858,6560835.70,7.2662,6.7834,8.1741,1.01,-0.106164,2015-01-06 09:30:00
2,000004,15.57,15.58,15.90,15.30,15.69,15.5709,28095.12,0.7707,3.3496,4.5343,0.12,0.006390,2015-01-06 09:30:00
3,000006,7.05,7.10,7.39,7.02,7.08,7.1881,1078647.12,0.4255,8.0562,13.2751,0.03,-0.078571,2015-01-06 09:30:00
4,000007,14.60,14.51,14.80,14.25,14.52,14.5105,28415.72,-0.5479,1.3807,1.7538,-0.08,-0.004844,2015-01-06 09:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4024,688699,237.77,285.32,285.32,285.32,285.32,285.3200,1225.82,19.9983,0.6940,0.6940,47.55,-0.024476,2021-07-01 09:30:00
4025,688777,90.00,89.95,94.80,89.06,94.15,92.9798,19001.61,4.6111,4.6288,4.6288,4.15,0.053133,2021-07-01 09:30:00
4026,688788,138.20,137.28,148.50,136.33,145.38,144.3683,13572.56,5.1954,7.7977,7.7977,7.18,-0.102721,2021-07-01 09:30:00
4027,688819,43.68,43.40,44.98,43.31,44.49,44.3727,47249.01,1.8544,4.6754,4.6754,0.81,0.032164,2021-07-01 09:30:00


In [ ]:
df.set_index(['timestamp','ticker']).to_csv(output_path + 'data.csv')
# Save_TimeStamp
df['timestamp'].drop_duplicates()

In [23]:
df_no_return
for i in range(1,2):
    display(df_no_return.shift(-i))

pre_close    open    high     low   close  \
timestamp           ticker                                              
2015-01-06 09:30:00 1           13.90   14.39   15.29   14.22   14.91   
                    2           15.57   15.58   15.90   15.30   15.69   
                    4            7.05    7.10    7.39    7.02    7.08   
                    6           14.60   14.51   14.80   14.25   14.52   
                    7           22.09   22.10   23.03   21.60   21.70   
...                               ...     ...     ...     ...     ...   
2021-07-01 09:30:00 688699      90.00   89.95   94.80   89.06   94.15   
                    688777     138.20  137.28  148.50  136.33  145.38   
                    688788      43.68   43.40   44.98   43.31   44.49   
                    688819      59.25   59.69   62.50   59.40   61.82   
                    688981        NaN     NaN     NaN     NaN     NaN   

                                vwap      volume  pct_chg  turnover  \
timestamp           ticker                                            
2015-01-06 09:30:00 1        14.7858  6560835.70   7.2662    6.7834   
                    2        15.5709    28095.12   0.7707    3.3496   
                    4         7.1881  1078647.12   0.4255    8.0562   
                    6        14.5105    28415.72  -0.5479    1.3807   
                    7        22.3283    22900.58  -1.7655    1.5546   
...                              ...         ...      ...       ...   
2021-07-01 09:30:00 688699   92.9798    19001.61   4.6111    4.6288   
                    688777  144.3683    13572.56   5.1954    7.7977   
                    688788   44.3727    47249.01   1.8544    4.6754   
                    688819   61.5420   777950.53   4.3376    7.1003   
                    688981       NaN         NaN      NaN       NaN   

                            free_turnover   chg  
timestamp           ticker                       
2015-01-06 09:30:00 1              8.1741  1.01  
                    2              4.5343  0.12  
                    4             13.2751  0.03  
                    6              1.7538 -0.08  
                    7              1.7862 -0.39  
...                                   ...   ...  
2021-07-01 09:30:00 688699         4.6288  4.15  
                    688777         7.7977  7.18  
                    688788         4.6754  0.81  
                    688819         7.1003  2.57  
                    688981            NaN   NaN  

[4851238 rows x 11 columns]

In [36]:
df2 = pd.read_csv("/home/wuwenjun/Data/AlphaNet_Original_Input.csv")
df2

,timestamp,ticker,pre_close,open,high,low,close,vwap,volume,pct_chg,turnover,free_turnover,chg,5d_ret
0,2015-01-06 09:30:00,1,15.84,15.99,16.28,15.60,16.02,15.9605,2860436.43,1.1364,2.9079,5.7657,0.18,-0.075710
1,2015-01-06 09:30:00,2,13.90,14.39,15.29,14.22,14.91,14.7858,6560835.70,7.2662,6.7834,8.1741,1.01,-0.106164
2,2015-01-06 09:30:00,4,15.57,15.58,15.90,15.30,15.69,15.5709,28095.12,0.7707,3.3496,4.5343,0.12,0.006390
3,2015-01-06 09:30:00,6,7.05,7.10,7.39,7.02,7.08,7.1881,1078647.12,0.4255,8.0562,13.2751,0.03,-0.078571
4,2015-01-06 09:30:00,7,14.60,14.51,14.80,14.25,14.52,14.5105,28415.72,-0.5479,1.3807,1.7538,-0.08,-0.004844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4851233,2021-07-01 09:30:00,688699,237.77,285.32,285.32,285.32,285.32,285.3200,1225.82,19.9983,0.6940,0.6940,47.55,-0.024476
4851234,2021-07-01 09:30:00,688777,90.00,89.95,94.80,89.06,94.15,92.9798,19001.61,4.6111,4.6288,4.6288,4.15,0.053133
4851235,2021-07-01 09:30:00,688788,138.20,137.28,148.50,136.33,145.38,144.3683,13572.56,5.1954,7.7977,7.7977,7.18,-0.102721
4851236,2021-07-01 09:30:00,688819,43.68,43.40,44.98,43.31,44.49,44.3727,47249.01,1.8544,4.6754,4.6754,0.81,0.032164


In [9]:
import multiprocessing as mp

In [37]:
for ticker in df2['ticker'].drop_duplicates()[:2]:
    one_data = df2[df2['ticker'] == ticker]
    dict_output = {}
    for i in range(0,1):
#     for i in range(0,one_data.shape[0] - day ,3): # 其中3 代表取数的步长，ex.每两天取一次数，步长为3
        time_stamp = str(pd.to_datetime(one_data.iloc[i+day-1]['timestamp']))
        x = one_data.iloc[i:i+day,:]
        y = one_data.iloc[i+day-1][return_value]
        x.set_index(['ticker','timestamp'],inplace=True)
        x.drop(target,axis=1,inplace=True)
        dict_output[time_stamp]= [x,y]
        display(x)
        print(y)
#     np.save('/home/wuwenjun/Data/AlphaNet_Original_Input/%s.npy'%ticker, dict_output) 
#     return None

,,pre_close,open,high,low,close,vwap,volume,pct_chg,turnover,free_turnover,chg
ticker,timestamp,,,,,,,,,,,


-0.001315212626041


,,pre_close,open,high,low,close,vwap,volume,pct_chg,turnover,free_turnover,chg
ticker,timestamp,,,,,,,,,,,


0.0108832147341981


In [55]:
# dict_output = {}
# for time_stamp in df['timestamp'].drop_duplicates():
#     time_stamp = str(time_stamp)
#     dict_output[time_stamp] = {}
# dict_output

In [40]:
df2[df2['ticker'] == 1].head(35).reset_index(drop=True)

,timestamp,ticker,pre_close,open,high,low,close,vwap,volume,pct_chg,turnover,free_turnover,chg,5d_ret
0,2015-01-06 09:30:00,1,15.84,15.99,16.28,15.60,16.02,15.9605,2860436.43,1.1364,2.9079,5.7657,0.18,-0.075710
1,2015-01-07 09:30:00,1,16.02,15.85,16.39,15.55,15.78,15.9408,2166421.40,-1.4981,2.2024,4.3668,-0.24,-0.050129
2,2015-01-08 09:30:00,1,15.78,15.56,15.83,15.30,15.48,15.4977,1700120.67,-1.9011,1.7283,3.4269,-0.30,-0.041935
3,2015-01-09 09:30:00,1,15.48,15.50,15.57,14.90,14.96,15.1167,1407714.21,-3.3592,1.4311,2.8375,-0.52,0.033557
4,2015-01-12 09:30:00,1,14.96,14.90,15.87,14.71,15.08,15.2895,2508500.23,0.8021,2.5501,5.0564,0.12,-0.057835
5,2015-01-13 09:30:00,1,15.08,14.87,15.05,14.50,14.77,14.7629,1553290.86,-2.0557,1.5791,3.1309,-0.31,-0.055973
6,2015-01-14 09:30:00,1,14.77,14.65,14.90,14.61,14.68,14.7512,816874.77,-0.6093,0.8304,1.6466,-0.09,-0.060893
7,2015-01-15 09:30:00,1,14.68,14.78,15.20,14.70,14.81,14.9585,1263029.64,0.8856,1.2840,2.5459,0.13,-0.034343
8,2015-01-16 09:30:00,1,14.81,14.85,15.35,14.71,15.35,15.0446,1242170.32,3.6462,1.2628,2.5038,0.54,-0.067532
9,2015-01-19 09:30:00,1,15.35,15.40,15.62,15.18,15.37,15.4472,1555846.33,0.1303,1.5817,3.1361,0.02,0.024982


In [37]:
def test(ticker):
    one_data = df[df['ticker'] == ticker]
    dict_output = {}
    for i in range(0,1):
#     for i in range(0,one_data.shape[0] - day ,3): # 其中3 代表取数的步长，ex.每两天取一次数，步长为3
        time_stamp = str(pd.to_datetime(one_data.iloc[i+day-1]['timestamp']))
        x = one_data.iloc[i:i+day,:]
        y = one_data.iloc[i+day-1][return_value]
        x.set_index(['ticker','timestamp'],inplace=True)
        dict_output[time_stamp]= [x,y]
        display(x)
#     np.save('/home/wuwenjun/Data/AlphaNet_Original_Input/%s.npy'%ticker, dict_output) 
    return None

In [51]:
for ticker in df['ticker'].drop_duplicates()[:1]:
    one_data = df[df['ticker'] == ticker]
    one_data.set_index(['timestamp','ticker'],inplace=True)
    target_df = one_data[target]
    one_data.drop(target,axis=1,inplace=True)
    concat_df = pd.DataFrame()
    for day in tqdm(range(29,-1,-1)):
        concat_df = pd.concat([concat_df,one_data.shift(day)],axis=1)
    concat_df.dropna(axis=0,how="any",inplace=True)
    concat_df = pd.concat([concat_df,target_df],axis=1,join='inner')

100%|██████████| 30/30 [00:00<00:00, 352.20it/s]


,,pre_close,open,high,low,close,vwap,volume,pct_chg,turnover,free_turnover,...,high,low,close,vwap,volume,pct_chg,turnover,free_turnover,chg,5d_ret
timestamp,ticker,,,,,,,,,,,,,,,,,,,,,
2015-02-16 09:30:00,1,15.84,15.99,16.28,15.60,16.02,15.9605,2860436.43,1.1364,2.9079,5.7657,...,14.18,13.84,13.95,14.0189,887743.10,0.6494,0.9025,1.7894,0.09,0.005735
2015-02-17 09:30:00,1,16.02,15.85,16.39,15.55,15.78,15.9408,2166421.40,-1.4981,2.2024,4.3668,...,14.01,13.73,13.92,13.8643,721202.39,-0.2151,0.7332,1.4537,-0.03,0.001433
2015-02-25 09:30:00,1,15.78,15.56,15.83,15.30,15.48,15.4977,1700120.67,-1.9011,1.7283,3.4269,...,14.11,13.93,13.99,14.0256,680491.59,0.5029,0.6918,1.3717,0.07,-0.029915
2015-02-26 09:30:00,1,15.48,15.50,15.57,14.90,14.96,15.1167,1407714.21,-3.3592,1.4311,2.8375,...,14.05,13.76,13.80,13.8414,577392.33,-1.3581,0.5870,1.1638,-0.19,-0.021739
2015-02-27 09:30:00,1,14.96,14.90,15.87,14.71,15.08,15.2895,2508500.23,0.8021,2.5501,5.0564,...,14.08,13.60,14.07,13.8859,1025544.24,1.9565,1.0426,2.0672,0.27,-0.049075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-25 09:30:00,1,23.55,23.52,23.59,22.84,23.07,23.1409,549652.50,-2.0382,0.2832,0.6390,...,23.12,22.68,23.08,22.9193,390283.49,-0.2593,0.2011,0.4538,-0.06,-0.015598
2021-06-28 09:30:00,1,22.89,23.14,23.43,22.60,23.32,23.0629,563785.36,1.8785,0.2905,0.6555,...,23.60,23.06,23.36,23.3991,559972.95,1.2132,0.2886,0.6510,0.28,-0.071490
2021-06-29 09:30:00,1,23.32,23.15,23.82,22.95,23.60,23.5347,527564.65,1.2007,0.2719,0.6134,...,23.39,22.48,22.78,22.7654,626257.37,-2.4829,0.3227,0.7281,-0.58,-0.022143


In [38]:
Process_jobs = []
if __name__ == '__main__':
#     for ticker in tqdm(df['ticker'].drop_duplicates()[:1]):
    t1 = time.time()
    pool = mp.Pool(70)  # 创建拥有3个进程数量的进程池
    pool.map(test, df['ticker'].drop_duplicates()[:1])
    pool.close()  # 关闭进程池，不再接受新的进程
    pool.join()  # 主进程阻塞等待子进程的退出
    t2 = time.time()
    print("并行执行时间：", int(t2 - t1))

并行执行时间： 11


In [59]:
read_dictionary = np.load('/home/wuwenjun/Data/AlphaNet_Original_Input/300965.npy',allow_pickle=True).item()

In [60]:
read_dictionary

{}

In [17]:
# Save

np.save('/home/wuwenjun/Data/AlphaNet_Original_Input.npy', dict_output) 

# Load
read_dictionary = np.load('my_file.npy').item()
# print(read_dictionary['hello']) 

KeyboardInterrupt: 

In [ ]:
dataframe_list.to_csv('/home/wuwenjun/Data/%s_%s.csv' % (time_start,time_end))
print("save to " + '/home/wuwenjun/Data/%s_%s.csv' % (time_start,time_end))